# HW12


In [23]:
import ast
# note this is python2 compatible only. the pykriging code is also python2 only
import urllib2
import urllib
from cStringIO import StringIO
import json
import geopandas as gpd
import pandas as pd
import shapely
from shapely.wkt import loads as load_wkt
from geopandas import GeoDataFrame
import os

In [2]:
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql'

def queryCartoDB(query, format = 'CSV', source = SQL_SOURCE):
    """ queries a url with an sql compatible search
    Arguments:
    query : the sql query string
    format : the expected format
    source " the url of the API
    """
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

In [3]:
# fill in the query details in facilityQuery: 
# within a single query collect AND GROUP the data by zipcode
facilityQuery = '''
SELECT
    count(*) as count,
    zipcodes
FROM nycasthmasdismissals 
GROUP BY        
    zipcodes
ORDER BY count DESC
'''

facilityStream = queryCartoDB(facilityQuery, format='JSON')
facilityData   = json.loads(facilityStream.read())
#facilityData
facilities     = pd.DataFrame.from_dict(facilityData['rows'])
facilities

,count,zipcodes
0,2164,10467
1,1829,11203
2,1699,10461
3,1698,10451
4,1156,11237
5,1072,10456
6,940,11201
7,926,11040
8,822,10457
9,713,11212


In [6]:
facilities['zipcodes'] = facilities['zipcodes'].astype(str)

In [29]:
#Get zip code shapes
shape_path=os.getenv('PUIDATA')+"/DOITT_ZIP_CODE_01_29JUL2009.shp"
#shape_path = "DOITT_ZIP_CODE_01_29JUL2009.shp"
#zip_shape = GeoDataFrame.from_file(shape_path)
#zip_shape.index = ct_shape.BoroCT2010
zip_shape.head()

IOError: Unable to open DOITT_ZIP_CODE_01_29JUL2009.shx or DOITT_ZIP_CODE_01_29JUL2009.SHX.Try --config SHAPE_RESTORE_SHX true to restore or create it

In [6]:
zip_shape['ZIP'] = zip_shape['ZIP'].astype(str)

In [7]:
zip_shape.rename(columns={'ZIP':'zipcodes'}, inplace=True)

In [8]:
#merge
facilitydb = pd.merge(facilities, zip_shape, how='inner', on=['zipcodes'])
facilitydb.drop(['BLDGZIP', 'COUNTY', 'CTY_FIPS', 'PO_NAME',
       'STATE', 'ST_FIPS'], axis=1, inplace=True)
facilitydb.head()

,count,zipcodes,Shape_Area,Shape_Leng,geometry
0,2164,10467,6.933617e+07,43944.750630,"POLYGON ((1017396.305742651 265444.5915357173,..."
1,1829,11203,6.064484e+07,32799.751802,"POLYGON ((1000147.787418231 181214.7914917171,..."
2,1699,10461,6.282406e+07,35943.350556,"POLYGON ((1027920.925769478 243665.9881113768,..."
3,1698,10451,2.894411e+07,32573.914438,"POLYGON ((1002809.989770979 240846.7194717228,..."
4,1156,11237,2.737378e+07,30772.617425,"POLYGON ((1004164.153073147 199490.0016603768,..."
